In [ ]:
import subprocess
import os
import time
import pandas as pd
from Bio import SeqIO
from Bio import pairwise2


from helpers.consensus_utils import select_best_alignment
from helpers.fastq import concatenate_fastq

In [ ]:
def pipeline_consensus(input_name: str, preprocess: bool = False, windows: bool = False): 
    input_fastq_path = os.path.join("assets", "input", input_name)

    # In case the fastq are not yet concatenated, we do so here before proceeding with the rest of the code
    if preprocess : 
        if not os.path.isdir(input_fastq_path) : 
            raise NotADirectoryError
        
        concatenate_fastq(input_fastq_path, input_fastq_path + ".fastq")

    input_fastq_path = input_fastq_path + ".fastq"
    if not os.path.isfile(input_fastq_path) : 
        raise FileNotFoundError
    
    output_dir = os.path.join("assets", "output", input_name)
    os.makedirs(output_dir, exist_ok=True)

    # Split the file into top 20% and remaining 80%
    top_sequences_path, remaining_sequences_path = split_fastq(input_fastq_path, output_dir, input_name, percentile=20)

    total_time_taken = 0

    # Step 1: Align and generate consensus from top 20% sequences
    top_paf_path = os.path.join(output_dir, f"{input_name}_top20_reads.paf")
    top_consensus_path = os.path.join(output_dir, f"{input_name}_top20_consensus.fasta")
    minimap2_command = f"minimap2 -x ava-ont {top_sequences_path} {top_sequences_path} > {top_paf_path}"
    if windows : 
        minimap2_command = minimap2_command.replace('\\','/')
    print("Running read alignment with minimap2 on top 20% sequences...")
    _, minimap2_time = run_command(minimap2_command, windows)
    total_time_taken += minimap2_time

    racon_command = f"racon -m 8 -x -6 -g -8 -w 500 {top_sequences_path} {top_paf_path} {top_sequences_path} > {top_consensus_path}"
    if windows : 
        racon_command = racon_command.replace('\\','/')
    print("Generating consensus sequence with racon on top 20% sequences...")
    _, racon_time = run_command(racon_command, windows)
    total_time_taken += racon_time

    # Step 1 bis: If the outputed racon file contains more than one sequence, we take the longest one
    consensus_sequences = list(SeqIO.parse(top_consensus_path, "fasta"))
    if len(consensus_sequences) > 1:
        print(f"Multiple sequences found in {top_consensus_path}. Selecting the best alignment...")
        best_sequence = select_best_alignment(consensus_sequences, consensus_sequences[0])  # Choose the first sequence as the reference
        SeqIO.write(best_sequence, top_consensus_path, "fasta")

    # Step 2: Align the remaining 80% sequences to the top 20% consensus
    remaining_paf_path = os.path.join(output_dir, f"{input_name}_remaining80_reads.paf")
    final_consensus_path = os.path.join(output_dir, f"{input_name}_final_consensus.fasta")
    minimap2_command = f"minimap2 -x map-ont {top_consensus_path} {remaining_sequences_path} > {remaining_paf_path}"
    if windows : 
        minimap2_command = minimap2_command.replace('\\', '/')
    print("Running read alignment with minimap2 on remaining 80% sequences...")
    _, minimap2_time = run_command(minimap2_command, windows)
    total_time_taken += minimap2_time

    # Step 3: Generate the final consensus sequence with racon
    racon_command = f"racon -m 8 -x -6 -g -8 -w 500 {remaining_sequences_path} {remaining_paf_path} {top_consensus_path} > {final_consensus_path}"
    if windows : 
        racon_command = racon_command.replace('\\','/')
    print("Generating final consensus sequence with racon...")
    _, racon_time = run_command(racon_command, windows)
    total_time_taken += racon_time

    # Print out the total time for each step
    print(f"Minimap2 alignment took {minimap2_time:.2f} seconds.")
    print(f"Total Racon iterations took {total_time_taken - minimap2_time:.2f} seconds.")

    # Print out the total time for the pipeline
    print(f"Total time taken for the pipeline: {total_time_taken:.2f} seconds.")


In [ ]:
def pipeline_identification(input_name: str, database: str = None, windows: bool = False) :
    input_fasta_path = os.path.join("assets", "output", input_name, input_name + "_final_consensus.fasta")
    output_blastn = os.path.join("assets", "output", "blastn", input_name)
    os.makedirs(output_blastn, exist_ok=True)

    if database == None : 
        for db in {"ITS_seqs", "matK_seqs", "psbA-trnH_seqs", "rbcL_seqs"} :
            output_blastn_path = os.path.join(output_blastn, db + ".txt")
            blastn_cmd = f"blastn -query {input_fasta_path} -db {db} -out {output_blastn_path} -max_target_seqs 5"
            if windows : 
                blastn_cmd = blastn_cmd.replace("\\", "/")
            run_command(blastn_cmd)
    else :
        output_blastn_path = os.path.join(output_blastn, database + ".txt")
        blastn_cmd = f"blastn -query {input_fasta_path} -db {database} -out {output_blastn_path} -max_target_seqs 5"
        if windows : 
                blastn_cmd = blastn_cmd.replace("\\", "/")
        run_command(blastn_cmd)

    print("You can find identification ouput at ", output_blastn)


In [ ]:
def main(input_name: str, preprocess: bool = False, database: str = None, windows: bool = False) :
    print("Debuting consensus sequencing : ") 
    pipeline_consensus(input_name, preprocess, windows)
    print("Consensus sequencing finished")
    print("Debuting Identification with blastn : ")
    pipeline_identification(input_name, database, windows)
    print("Identification finished")

In [ ]:
main("rbcL_Qiagen_tomato_5000", False, "rbcL_seqs", True)